In [2]:
import cPickle as pickle
import numpy as np

import os
import os.path as osp
import skimage.io as sio
import skimage.transform


# from natsort import natsorted

In [3]:
root = '/home/hoa/Desktop/projects/resources'

In [4]:
im_labels = np.load(osp.join(root,'datasets/msrcv2/imgsWithLabels.npy')).item()

In [ ]:
im_names = sorted(im_labels.keys())

In [ ]:
label_color = np.load(osp.join(root,'datasets/msrcv2/labelCodes.npy')).item()
label_color

# Create X

In [ ]:
N = len(im_labels); K = len(label_color)

n_ops = 5
Xaug = np.zeros((N*n_ops, 227, 227, 3), dtype='uint8')

In [9]:
def augment_image(im, im_name=""):  
    out = np.zeros((5, 227, 227, 3))
    h,w,_ = im.shape
    #print h,w
    
    # resized if not done yet    
    if h is not 256 and w is not 256:
#         print 'Resizing',im_name
        if h>=w:
            im = skimage.transform.resize(im,
                (h*256//w, 256), 
                mode='nearest', preserve_range=True)    
        else:
            im = skimage.transform.resize(im,
                (256, w*256//h), 
                mode='nearest', preserve_range=True)    
    
    h,w,_ = im.shape
    #print h,w
    # 5 crops
    out[0] = im[h//2-113:h//2+114, w//2-113:w//2+114, :]
    out[1] = im[:227, :227, :]
    out[2] = im[-227:, :227, :]
    out[3] = im[:227, -227:, :]
    out[4] = im[-227:, -227:, :]
    
    return out

In [ ]:
path = osp.join(root,'datasets/msrcv2/Images')

In [ ]:
for n in xrange(N):
    im_name = im_names[n]
    im = sio.imread(osp.join(path,im_name)+'.bmp')

    out = augment_image(im, im_name)
    for i in xrange(n_ops):
        Xaug[n+N*i]=out[i]

In [ ]:
Xaug.shape

In [ ]:
np.save(osp.join(root,'datasets/msrcv2/Xaug_b01c.npy'), Xaug)

## Create Y

Note that `void` is still included in `im_labels` with id=22 (23rd column), so we will create Y of size `N x (K+1)` and remove column index 22

In [ ]:
Y = np.zeros((N,K+1), dtype='int32') 
for i in xrange(len(im_names)):    
    col_indices = np.asarray(im_labels[im_names[i]])-1
    
    if np.all(col_indices>=0):
        Y[i, col_indices]=1
    else:
        print 'Error: negative column index'
        
Y = np.delete(Y,22,1)
Y.shape

In [ ]:
Yaug = np.tile(Y, (n_ops,1))

In [ ]:
Yaug.shape

In [ ]:
np.save(osp.join(root,'datasets/msrcv2/Y.npy'), Y)

In [ ]:
np.save(osp.join(root,'datasets/msrcv2/Yaug.npy'), Yaug)

***Double check if image augmentation works properly***

In [ ]:
Xaug = np.load(osp.join(root,'datasets/msrcv2/Xaug_b01c.npy'))
Yaug = np.load(osp.join(root,'datasets/msrcv2/Yaug.npy'))

import matplotlib.pylab as plt
%matplotlib inline

idx = np.random.randint(0,len(Xaug),20)
label_names = np.asarray(sorted(label_color.keys()))

def plot_image(im):
    plt.figure()
    plt.imshow(im)
    plt.axis('off')    

for i in idx:
    plot_image(Xaug[i])
    
    rawid = i%N
    plt.text(256,30,str(im_names[rawid])+': '+str(label_names[np.where(Y[rawid]==1)]))
    

## for Corel5K

In [ ]:
X_train = np.load(osp.join(root,'datasets/corel5k/X_train_rgb.npy'))

In [7]:
n_ops=5
Xaug = np.zeros((N*n_ops, 227, 227, 3), dtype='uint8')
N=len(X_train)

In [11]:
for n in xrange(N):    
    im = X_train[n]

    out = augment_image(im)
    for i in xrange(n_ops):
        Xaug[n+N*i]=out[i]

In [5]:
Y_test = np.load(osp.join(root,'datasets/corel5k/Y_test.npy'))

In [6]:
len(Y_test)

499

In [ ]:
N = Yaug.shape[0]

In [ ]:
import matplotlib.pylab as plt
%matplotlib inline

idx = np.random.randint(0,len(Xaug),10)
# label_names = np.asarray(sorted(label_color.keys()))

def plot_image(im):
    plt.figure()
    plt.imshow(im)
    plt.axis('off')    

for i in idx:
    plot_image(Xaug[i])
    
    rawid = i%N
#     plt.text(256,30,str(im_names[rawid])+': '+str(label_names[np.where(Y[rawid]==1)]))
    